# Challenge WILLIAM BENHAIM

## Partie principale
Lancer le code jusqu'à la partie annexe permet de générer mon code fichier de prediction final.

In [1]:
# Import des librairies utiles
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import preprocessing
import time
import datetime
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import Imputer
from sklearn.metrics import roc_auc_score
from sklearn import ensemble
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from sklearn.datasets import load_digits
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.preprocessing import Imputer
from sklearn.cross_validation import KFold

# On recupere les donnees
train_fname = '/Users/williambenhaim/Desktop/TelecomParisTech/P2/MachineLearning/Challenge/train.csv'
test_fname = '/Users/williambenhaim/Desktop/TelecomParisTech/P2/MachineLearning/Challenge/test.csv'

# Stockage des donnees, on remplace les 'MISSING' par des Nan
df_ini = pd.read_csv(train_fname, sep=';', na_values='(MISSING)')
df = df_ini.drop(['VARIABLE_CIBLE'], axis=1)
df_test = pd.read_csv(test_fname, sep=';', na_values='(MISSING)')

# Creation d'une dataset avec toutes les observations disponibles.
frames = [df, df_test]
df_concat = pd.concat(frames)

y_train = df_ini.VARIABLE_CIBLE == 'GRANTED'

# Tous les types de donnees numeriques
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']

# Recuperation du nom des colonnes pour les dates, les valeurs numeriques et nom numeriques
nonNumericColumns = []
numericColumns = []
dateColumns = ['PRIORITY_MONTH', 'FILING_MONTH', 'PUBLICATION_MONTH', 'BEGIN_MONTH']
numericColumns = df_concat.select_dtypes(include=numerics).columns.difference(dateColumns)
nonNumericColumns = df_concat.columns.difference(dateColumns).difference(numericColumns)

# On remplace les Nan par la derniere observation connue, et on recupere que l'annee
for col in dateColumns:
    df[col] = df[col].fillna(method="bfill")
    df_test[col] = df_test[col].fillna(method="bfill")
    df[col] = pd.DataFrame(df[col].str.split('/').str[1])
    df_test[col] = pd.DataFrame(df_test[col].str.split('/').str[1])

# On transforme les donnees textuelles en label, on concidere les Nan comme des informations
# a part entiere
le = preprocessing.LabelEncoder()
for column in nonNumericColumns:
    # On fit sur le train + test
    le.fit(df_concat[column])
    # On applique sur le train puis sur le test
    df[column] = le.transform(df[column])
    df_test[column] = le.transform(df_test[column])

# On remplace les Nan par la moyenne des observations pour les donnees numeriques
imp = Imputer(missing_values='NaN', strategy='mean', axis=0)
imp.fit(df_concat[numericColumns])
df[numericColumns] = imp.transform(df[numericColumns])
df_test[numericColumns] = imp.transform(df_test[numericColumns])

feature_names = df_concat.columns
X_train = df[feature_names].values
X_test = df_test[feature_names].values

# Paremetres trouves avec le Hyperopt en annexe et valides avec par cross validation
# Moyenne des deux meilleurs algorithmes pour generer le fichier final
paramsA = {'reg_alpha': 1.7550000000000001, 'colsample_bytree': 0.45, 'learning_rate': 0.0225,
        'nthread': -1, 'min_child_weight': 3.0, 'n_estimators': 779, 'subsample': 0.9500000000000001, 
        'reg_lambda': 0.11, 'objective': 'binary:logistic', 'max_depth': 11, 'gamma': 1.0}

paramsB = {'reg_alpha': 0.06, 'colsample_bytree': 0.55, 'learning_rate': 0.0125,
          'nthread': -1, 'min_child_weight': 3.0, 'n_estimators': 1723, 
          'subsample': 0.9500000000000001, 'reg_lambda': 0.38, 'objective': 'binary:logistic',
          'max_depth': 10, 'gamma': 0.9500000000000001}

gbmA = xgb.XGBClassifier(**paramsA)
gbmA.fit(X_train, y_train, eval_metric="auc")
y_predA = gbmA.predict_proba(X_test)[:, 1]

gbmB = xgb.XGBClassifier(**paramsB)
gbmB.fit(X_train, y_train, eval_metric="auc")
y_predB = gbmB.predict_proba(X_test)[:, 1]
# On genere le fichier de prediction final
y_pred = (y_predA + y_predB) / 2
np.savetxt('BENHAIM final.txt', y_pred, fmt='%s')
print 'done'


/Users/williambenhaim/anaconda/lib/python2.7/site-packages/numpy/lib/arraysetops.py:200: FutureWarning: numpy not_equal will not check object identity in the future. The comparison did not return the same result as suggested by the identity (`is`)) and will change.
  flag = np.concatenate(([True], aux[1:] != aux[:-1]))
/Users/williambenhaim/anaconda/lib/python2.7/site-packages/numpy/lib/arraysetops.py:259: FutureWarning: numpy equal will not check object identity in the future. The comparison did not return the same result as suggested by the identity (`is`)) and will change.
  return aux[:-1][aux[1:] == aux[:-1]]


done


## Code Annexe
Ce code comprend un hyperopt sur XGBOOST, un RandomizeCV sur RandomForest, ainsi que ma derniere cross validation.


In [4]:
paramsA = {'reg_alpha': 1.7550000000000001, 'colsample_bytree': 0.45, 'learning_rate': 0.0225,
        'nthread': -1, 'min_child_weight': 3.0, 'n_estimators': 779, 'subsample': 0.9500000000000001, 
        'reg_lambda': 0.11, 'objective': 'binary:logistic', 'max_depth': 11, 'gamma': 1.0}

paramsB = {'reg_alpha': 0.06, 'colsample_bytree': 0.55, 'learning_rate': 0.0125,
          'nthread': -1, 'min_child_weight': 3.0, 'n_estimators': 1723, 
          'subsample': 0.9500000000000001, 'reg_lambda': 0.38, 'objective': 'binary:logistic', 
          'max_depth': 10, 'gamma': 0.9500000000000001}


def cv_estimate(n_folds=5):
    cv = KFold(n=X_train.shape[0], n_folds=n_folds)
    gbm1 = xgb.XGBClassifier(**paramsA)
    gbm2 = xgb.XGBClassifier(**paramsB)
    val_scores = 0
    for train, test in cv:
        gbm1.fit(X_train[train], y_train[train], eval_metric="auc")
        gbm2.fit(X_train[train], y_train[train], eval_metric="auc")
        y_pred_train1 = gbm1.predict_proba(X_train[test])[:, 1]
        y_pred_train2 = gbm2.predict_proba(X_train[test])[:, 1]
        y_pred_train = (y_pred_train1 + y_pred_train2) / 2
        val_scores +=  roc_auc_score(y_train[test], y_pred_train)
    val_scores /= n_folds
    return val_scores

print cv_estimate()


0.717519060324


In [6]:
## https://github.com/bamine/Kaggle-stuff/blob/master/otto/hyperopt_xgboost.py
from sklearn.cross_validation import train_test_split
a_train, a_test, b_train, b_test = train_test_split(X_train, y_train, test_size=0.20)

space4rf = {
            'colsample_bytree' : hp.quniform('colsample_bytree', 0.3, 0.7, 0.05),
            'gamma' : hp.quniform('gamma', 0.7, 1, 0.05),
            'learning_rate' : hp.quniform('eta', 0.0001, 0.04, 0.0005),
            'max_depth' : hp.choice('max_depth', range(8, 12)),
            'min_child_weight' : hp.quniform('min_child_weight', 2., 6., 1),
            'n_estimators': 2000,
            'nthread':-1,
            'objective':'binary:logistic',
            'reg_alpha':hp.quniform('reg_alpha', 0.0, 5.0, 0.01),
            'reg_lambda':hp.quniform('reg_lambda', 0.0, 5.0, 0.01),
            'subsample' : hp.quniform('subsample', 0.7, 1, 0.05),
             }
best = 0
df_result_hyperopt = pd.DataFrame(columns=[np.append('score', (space4rf.keys()))])
i = 0


def auc_score(params):
    global i
    print '------------------'
    print i
    print params
    gbm = xgb.XGBClassifier(**params).fit(a_train, b_train, eval_metric="auc",early_stopping_rounds=200
                                          ,eval_set=[(a_test, b_test)])
    y_pred_train = gbm.predict_proba(a_test)[:, 1]
    loss = roc_auc_score(b_test, y_pred_train)
    df_result_hyperopt.columns=[np.append('score', (params.keys()))]
    df_result_hyperopt.loc[i] = np.append(loss, params.values())
    print loss
    print '------------------'
    i = i+1
    return {'loss': 1-loss, 'status': STATUS_OK}

trials = Trials()
best = fmin(auc_score, space4rf, algo=tpe.suggest, max_evals=5, trials=trials)



Will train until validation_0 error hasn't decreased in 200 rounds.
[0]	validation_0-auc:0.664713
[1]	validation_0-auc:0.675888
[2]	validation_0-auc:0.681312
[3]	validation_0-auc:0.684669
[4]	validation_0-auc:0.686017
[5]	validation_0-auc:0.687563
[6]	validation_0-auc:0.687924
[7]	validation_0-auc:0.687675
[8]	validation_0-auc:0.688294
[9]	validation_0-auc:0.689081
[10]	validation_0-auc:0.689012
[11]	validation_0-auc:0.688930
[12]	validation_0-auc:0.689467
[13]	validation_0-auc:0.690312
[14]	validation_0-auc:0.691226
[15]	validation_0-auc:0.691422
[16]	validation_0-auc:0.692140
[17]	validation_0-auc:0.692016
[18]	validation_0-auc:0.692402
[19]	validation_0-auc:0.693005
[20]	validation_0-auc:0.693038
[21]	validation_0-auc:0.693464
[22]	validation_0-auc:0.693506
[23]	validation_0-auc:0.693812
[24]	validation_0-auc:0.694101
[25]	validation_0-auc:0.694323
[26]	validation_0-auc:0.695031
[27]	validation_0-auc:0.695374
[28]	validation_0-auc:0.695950
[29]	validation_0-auc:0.696035
[30]	validat

------------------
0
{'reg_alpha': 0.5, 'colsample_bytree': 0.55, 'learning_rate': 0.0185, 'nthread': -1, 'min_child_weight': 4.0, 'n_estimators': 2000, 'subsample': 0.8, 'reg_lambda': 2.22, 'objective': 'binary:logistic', 'max_depth': 9, 'gamma': 0.8}
0.717658054364

Stopping. Best iteration:
[1330]	validation_0-auc:0.717724

Will train until validation_0 error hasn't decreased in 200 rounds.
[0]	validation_0-auc:0.666907
[1]	validation_0-auc:0.674777
[2]	validation_0-auc:0.677713
[3]	validation_0-auc:0.681575
[4]	validation_0-auc:0.684455
[5]	validation_0-auc:0.686867
[6]	validation_0-auc:0.686906
[7]	validation_0-auc:0.686942
[8]	validation_0-auc:0.687545
[9]	validation_0-auc:0.687858
[10]	validation_0-auc:0.688327
[11]	validation_0-auc:0.688111
[12]	validation_0-auc:0.688423
[13]	validation_0-auc:0.688938
[14]	validation_0-auc:0.688979
[15]	validation_0-auc:0.688928
[16]	validation_0-auc:0.689005
[17]	validation_0-auc:0.689159
[18]	validation_0-auc:0.689061
[19]	validation_0-auc:0.689265
[20]	validation_0-auc:0.689189
[21]	validation_0-auc:0.689423
[22]	validation_0-auc:0.689481
[23]	validation_0-auc:0.689495
[24]	validation_0-auc:0.689580
[25]	validation_0-auc:0.689694
[26]	validation_0-auc:0.689812
[27]	validation_0-auc:0.689951
[28]	validatio


------------------
------------------
1
{'reg_alpha': 2.91, 'colsample_bytree': 0.7000000000000001, 'learning_rate': 0.008, 'nthread': -1, 'min_child_weight': 4.0, 'n_estimators': 2000, 'subsample': 0.8500000000000001, 'reg_lambda': 2.91, 'objective': 'binary:logistic', 'max_depth': 9, 'gamma': 0.8500000000000001}
0.716648316169

Will train until validation_0 error hasn't decreased in 200 rounds.
[0]	validation_0-auc:0.652083
[1]	validation_0-auc:0.668910
[2]	validation_0-auc:0.670950
[3]	validation_0-auc:0.675747
[4]	validation_0-auc:0.678075
[5]	validation_0-auc:0.681824
[6]	validation_0-auc:0.680386
[7]	validation_0-auc:0.681484
[8]	validation_0-auc:0.682543
[9]	validation_0-auc:0.682509
[10]	validation_0-auc:0.684052
[11]	validation_0-auc:0.687015
[12]	validation_0-auc:0.686497
[13]	validation_0-auc:0.686557
[14]	validation_0-auc:0.688427
[15]	validation_0-auc:0.688091
[16]	validation_0-auc:0.688172
[17]	validation_0-auc:0.689028
[18]	validation_0-auc:0.689445
[19]	validation_0-auc:0.689346
[20]	validation_0-auc:0.689292
[21]	validation_0-auc:0.689795
[22]	validation_0-auc:0.690529
[23]	validation_0-auc:0.691379
[24]	validation_0-auc:0.691307
[25]	validation_0-auc:0.691680
[26]	validation_0-auc:0.692091
[27]	validation_0-auc:0.692476
[28]	validation_0-auc:0.692831
[29]	validation_0-auc:0.693465
[30]	validat

...

Stopping. Best iteration:
[1186]	validation_0-auc:0.717093

Will train until validation_0 error hasn't decreased in 200 rounds.
[0]	validation_0-auc:0.667703
[1]	validation_0-auc:0.678178
[2]	validation_0-auc:0.684267
[3]	validation_0-auc:0.688405
[4]	validation_0-auc:0.690418
[5]	validation_0-auc:0.692137
[6]	validation_0-auc:0.692242
[7]	validation_0-auc:0.692033
[8]	validation_0-auc:0.692517
[9]	validation_0-auc:0.693420
[10]	validation_0-auc:0.693583
[11]	validation_0-auc:0.693578
[12]	validation_0-auc:0.694325
[13]	validation_0-auc:0.695483
[14]	validation_0-auc:0.696281
[15]	validation_0-auc:0.696610
[16]	validation_0-auc:0.697272
[17]	validation_0-auc:0.697389
[18]	validation_0-auc:0.697738
[19]	validation_0-auc:0.698301
[20]	validation_0-auc:0.698512
[21]	validation_0-auc:0.698956
[22]	validation_0-auc:0.698931
[23]	validation_0-auc:0.699357
[24]	validation_0-auc:0.699613
[25]	validation_0-auc:0.699565
[26]	validation_0-auc:0.700246
[27]	validation_0-auc:0.700373
[28]	validatio


------------------
------------------
3
{'reg_alpha': 3.87, 'colsample_bytree': 0.55, 'learning_rate': 0.033, 'nthread': -1, 'min_child_weight': 6.0, 'n_estimators': 2000, 'subsample': 0.9500000000000001, 'reg_lambda': 0.71, 'objective': 'binary:logistic', 'max_depth': 10, 'gamma': 0.9}
0.717244008174

Stopping. Best iteration:
[687]	validation_0-auc:0.717490

Will train until validation_0 error hasn't decreased in 200 rounds.
[0]	validation_0-auc:0.664809
[1]	validation_0-auc:0.675483
[2]	validation_0-auc:0.680075
[3]	validation_0-auc:0.682497
[4]	validation_0-auc:0.684420
[5]	validation_0-auc:0.685623
[6]	validation_0-auc:0.686491
[7]	validation_0-auc:0.686045
[8]	validation_0-auc:0.687085
[9]	validation_0-auc:0.687868
[10]	validation_0-auc:0.688399
[11]	validation_0-auc:0.688279
[12]	validation_0-auc:0.689068
[13]	validation_0-auc:0.689891
[14]	validation_0-auc:0.690396
[15]	validation_0-auc:0.690606
[16]	validation_0-auc:0.691063
[17]	validation_0-auc:0.691201
[18]	validation_0-auc:0.691283
[19]	validation_0-auc:0.691711
[20]	validation_0-auc:0.691606
[21]	validation_0-auc:0.691876
[22]	validation_0-auc:0.691969
[23]	validation_0-auc:0.692497
[24]	validation_0-auc:0.692663
[25]	validation_0-auc:0.692876
[26]	validation_0-auc:0.693540
[27]	validation_0-auc:0.693847
[28]	validation


------------------
------------------
4
{'reg_alpha': 2.71, 'colsample_bytree': 0.6000000000000001, 'learning_rate': 0.017, 'nthread': -1, 'min_child_weight': 3.0, 'n_estimators': 2000, 'subsample': 0.75, 'reg_lambda': 0.66, 'objective': 'binary:logistic', 'max_depth': 9, 'gamma': 0.75}
0.71718643865
------------------


Stopping. Best iteration:
[1345]	validation_0-auc:0.717241



In [7]:
#J ai lancz unz derniere fois un hyperopt pour envoyer le challenge, mais je 
# n ai pas eu le temps de cross valider ces resultats qui m'ont l air tres interessant...
df_result_hyperopt = df_result_hyperopt.sort_values(
    'score', axis = 0, ascending=[False])
df_result_hyperopt.head()

,score,reg_alpha,colsample_bytree,learning_rate,nthread,min_child_weight,n_estimators,subsample,reg_lambda,objective,max_depth,gamma
0,0.717658054364,0.5,0.55,0.0185,-1,4.0,2000,0.8,2.22,binary:logistic,9,0.8
3,0.717244008174,3.87,0.55,0.033,-1,6.0,2000,0.95,0.71,binary:logistic,10,0.9
4,0.71718643865,2.71,0.6,0.017,-1,3.0,2000,0.75,0.66,binary:logistic,9,0.75
2,0.716843629316,0.95,0.4,0.0315,-1,6.0,2000,1.0,0.55,binary:logistic,8,0.9
1,0.716648316169,2.91,0.7,0.008,-1,4.0,2000,0.85,2.91,binary:logistic,9,0.85


In [ ]:
# build a classifier
clfRandom = RandomForestClassifier(min_samples_leaf = 10, min_samples_split = 3, n_jobs=-1)

# Utility function to report best scores
def report(grid_scores, n_top=3):
    top_scores = sorted(grid_scores, key=itemgetter(1), reverse=True)[:n_top]
    for i, score in enumerate(top_scores):
        print("Model with rank: {0}".format(i + 1))
        print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
              score.mean_validation_score,
              np.std(score.cv_validation_scores)))
        print("Parameters: {0}".format(score.parameters))
        print("")


# specify parameters and distributions to sample from
param_dist = {"n_estimators":sp_randint(1000, 2500),
              "max_features": sp_randint(6, 20),
              "max_depth":sp_randint(6, 25)
             }

# run randomized search
n_iter_search = 10
random_search = RandomizedSearchCV(clfRandom, scoring ='roc_auc',
                                   param_distributions=param_dist,
                                   n_iter=n_iter_search,n_jobs=-1)

start = time()
random_search.fit(a_train, b_train)
print("RandomizedSearchCV took %.2f seconds for %d candidates"
      " parameter settings." % ((time() - start), n_iter_search))
report(random_search.grid_scores_)
